#  Zillow Project: Data Preparation
---

## MVP Objectives
- Remove/Impute missing values.
- Create functions to reproduce prepared dataset
- Move functions to a seperate file named `prepare.py`


In [1]:
# Import libraries to manipulate data structures and visualize numeric data.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in the zillow dataset and assign to a variable
df = pd.read_csv('./data/raw/zillow.csv')

In [3]:
# My first dataset with +2 million rows :)
df.shape

(2985217, 59)

In [4]:
# Initial look at our zillow dataset
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


Our dataset has a bunch of missing values! This is only the first 5 rows! Let's see how many values are missing in each column.

In [5]:
# Using `isnull()` and `notnull()` we can calculate the number of missing values and non-null values.
nulls = df.isnull().sum()
non_nulls = df.notnull().sum()

# Add missing values and non-null values together to get the total number values in each column.
total_values = nulls + non_nulls

# Create a variable to store the percentage of missing values in each column.
pct_missing = (nulls/total_values).sort_values(ascending=False)

# Perform formatting to clearly see the percentage of missing values in each column.
pct_missing_chart = pct_missing.apply("{0:.2%}".format)

# Display table to the user showing the percentage of missing values in each column.
print('Percentage of values missing per column')
print('-' * 39)
print(f"{pct_missing_chart}")

Percentage of values missing per column
---------------------------------------
storytypeid                     99.95%
basementsqft                    99.95%
yardbuildingsqft26              99.91%
fireplaceflag                   99.83%
architecturalstyletypeid        99.80%
typeconstructiontypeid          99.77%
finishedsquarefeet13            99.74%
buildingclasstypeid             99.57%
pooltypeid10                    99.43%
decktypeid                      99.42%
finishedsquarefeet6             99.28%
poolsizesum                     99.06%
pooltypeid2                     98.89%
hashottuborspa                  98.32%
taxdelinquencyflag              98.11%
taxdelinquencyyear              98.11%
yardbuildingsqft17              97.27%
finishedsquarefeet15            93.63%
finishedsquarefeet50            93.17%
finishedfloor1squarefeet        93.17%
fireplacecnt                    89.51%
threequarterbathnbr             89.40%
pooltypeid7                     83.05%
poolcnt                

Before we move ahead let's drop columns with more than __9.14%__ of values missing. Imputing values in columns with > 9.14% of values missing is a waste of time. We need to make our dataset _Robust™_.
- If we can impute values in columns: `lotsizesquarefeet`, `finishedsquarefeet12`, great. If not, we can drop them.

In [6]:
columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()

In [7]:
df_subset = df.drop(columns=columns_to_drop)

In [8]:
# My eyes are happy
df_subset.sample(10)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
1729157,1729157,14352719,5.0,5.0,5.0,3838.0,3838.0,6059.0,5.0,33732024.0,...,1286.0,96939.0,0.0,2001.0,693427.0,1675162.0,2016.0,981735.0,20511.38,6.059052e+13
2427250,2427250,14133540,4.0,5.0,4.0,2472.0,2472.0,6059.0,4.0,33881859.0,...,1286.0,97041.0,6.0,1976.0,192089.0,379966.0,2016.0,187877.0,4462.34,6.059111e+13
1204661,1204661,14190779,3.0,4.0,3.0,2490.0,2490.0,6059.0,3.0,33869467.0,...,1286.0,97078.0,8.0,1976.0,211009.0,692407.0,2016.0,481398.0,7964.58,6.059022e+13
1883260,1883260,11481791,2.0,3.0,2.0,1340.0,1340.0,6037.0,2.0,33914031.0,...,3101.0,96107.0,0.0,1952.0,94665.0,271863.0,2016.0,177198.0,3824.64,6.037602e+13
2430826,2430826,13130310,0.0,0.0,NaN,2112.0,2112.0,6037.0,NaN,34238623.0,...,3101.0,96339.0,0.0,1978.0,NaN,NaN,2016.0,NaN,537.83,NaN
2804828,2804828,10750420,0.0,0.0,NaN,270.0,270.0,6037.0,NaN,34120821.0,...,3101.0,96336.0,0.0,1979.0,28642.0,117038.0,2016.0,88396.0,1967.77,6.037800e+13
1291555,1291555,12767507,1.0,3.0,1.0,1014.0,1014.0,6037.0,1.0,34048154.0,...,3101.0,96479.0,0.0,1947.0,20893.0,30804.0,2016.0,9911.0,773.89,6.037434e+13
1909611,1909611,17092589,3.0,4.0,3.0,2135.0,2135.0,6111.0,3.0,34280804.0,...,2061.0,97084.0,8.0,1994.0,229000.0,653000.0,2016.0,424000.0,6851.78,6.111001e+13
2851024,2851024,11119117,3.0,3.0,3.0,1951.0,1951.0,6037.0,3.0,34435834.0,...,3101.0,96377.0,0.0,1989.0,377064.0,532467.0,2016.0,155403.0,7710.98,6.037920e+13
1931759,1931759,12477463,2.0,2.0,2.0,978.0,978.0,6037.0,2.0,33876100.0,...,3101.0,96208.0,0.0,1981.0,124522.0,179912.0,2016.0,55390.0,2309.19,6.037554e+13


Sweet, our data looks so much better. __We freed up $\approx$ .7+ GB of memory__! Let's press forward.

In [9]:
original_mem_usage = df.memory_usage(index=False).sum()/1_000_000_000
subset_mem_usage = df_subset.memory_usage(index=False).sum()/1_000_000_000

free_memory = original_mem_usage - subset_mem_usage
print(f"{free_memory:.2f}GB of free memory!")

0.79GB of free memory!


### Subset of Zillow data
- Dropped columns missing more than 9.14% of values

In [10]:
# Will return. Retrieving and reproducing data dictionary for these columns.
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            int64  
 1   parcelid                      int64  
 2   bathroomcnt                   float64
 3   bedroomcnt                    float64
 4   calculatedbathnbr             float64
 5   calculatedfinishedsquarefeet  float64
 6   finishedsquarefeet12          float64
 7   fips                          float64
 8   fullbathcnt                   float64
 9   latitude                      float64
 10  longitude                     float64
 11  lotsizesquarefeet             float64
 12  propertycountylandusecode     object 
 13  propertylandusetypeid         float64
 14  rawcensustractandblock        float64
 15  regionidcity                  float64
 16  regionidcounty                float64
 17  regionidzip                   float64
 18  roomcnt               

In [11]:
# Create a function to clean our Zillow dataset
# Note: Create a wrangle function to acquire and prepare the dataset
def prepare_zillow(df):
    '''
    Signature: prepare_zillow(df) -> pandas.core.frame.DataFrame
    Docstring:
    Prepare the zillow dataset for data EDA

    Return DataFrame of zillow dataset


    Parameters
    ----------
    df : pandas.core.frame.DataFrame
    df is the Zillow dataset stored as `zillow.csv`


    Returns
    -------
    DataFrame of the zillow dataset


    Examples
    --------
    To copy the code hold ALT + SHIFT and drag your cursor from the first line to the last.

    >>> df = pd.read_csv('./data/raw/zillow.csv')
    >>>
    >>> nulls = df.isnull().sum()
    >>> non_nulls = df.notnull().sum()
    >>> total_values = nulls + non_nulls
    >>>
    >>> pct_missing = (nulls/total_values).sort_values(ascending=False)
    >>> pct_missing_chart = pct_missing.apply("{0:.2%}".format)
    >>>
    >>> print('Percentage of values missing per column')
    >>> print('-' * 39)
    >>> print(f"{pct_missing_chart}")
    '''
    
    # 1. Drop columns with missing values
    # Calculate the number of missing values and non-null values.
    nulls = df.isnull().sum()
    non_nulls = df.notnull().sum()

    # Get the total number values in each column.
    total_values = nulls + non_nulls

    # Create a variable to store the percentage of missing values in each column.
    # .3358 is a hard coded value from the original analysis.
    # To reproduce use the code in the docstring
    pct_missing = nulls/total_values
    columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()
    
    # drop columns missing more than 33.58% of data.
    df_cleaned = df.drop(columns=columns_to_drop)
    
    return df_cleaned

In [12]:
test = prepare_zillow(df)

In [13]:
# Sweet, the function works!
test.sample(10)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
2969413,2969413,10898502,4.0,8.0,4.0,3883.0,NaN,6037.0,4.0,34171029.0,...,3101.0,96446.0,0.0,1950.0,171508.0,246749.0,2016.0,75241.0,3276.70,6.037125e+13
1723090,1723090,12482309,1.0,3.0,1.0,1041.0,1041.0,6037.0,1.0,33854621.0,...,3101.0,96212.0,0.0,1950.0,45961.0,67260.0,2016.0,21299.0,1080.11,6.037571e+13
443574,443574,13985568,2.5,5.0,2.5,2604.0,2604.0,6059.0,2.0,33827605.0,...,1286.0,97023.0,8.0,1964.0,128335.0,344151.0,2016.0,215816.0,4131.42,6.059087e+13
9872,9872,12997933,5.0,5.0,5.0,2617.0,2617.0,6037.0,5.0,34103274.0,...,3101.0,96267.0,0.0,1950.0,242219.0,396730.0,2016.0,154511.0,5417.35,6.037432e+13
313555,313555,17118634,2.0,4.0,2.0,2055.0,2055.0,6111.0,2.0,34233345.0,...,2061.0,97107.0,8.0,1989.0,209553.0,419106.0,2016.0,209553.0,3447.00,6.111003e+13
2496433,2496433,12086682,1.0,2.0,1.0,966.0,966.0,6037.0,1.0,34139894.0,...,3101.0,96324.0,0.0,1923.0,55050.0,275269.0,2016.0,220219.0,3009.08,6.037302e+13
1769303,1769303,12841586,2.0,3.0,2.0,1150.0,1150.0,6037.0,2.0,33974630.0,...,3101.0,96492.0,0.0,1960.0,71931.0,268537.0,2016.0,196606.0,3456.77,6.037409e+13
2757432,2757432,12140308,3.0,3.0,3.0,1969.0,1969.0,6037.0,3.0,34131100.0,...,3101.0,96267.0,0.0,2002.0,262133.0,641158.0,2016.0,379025.0,7547.98,6.037431e+13
1372210,1372210,13118639,0.0,0.0,NaN,600.0,600.0,6037.0,NaN,34417484.0,...,3101.0,96401.0,0.0,1979.0,NaN,NaN,2016.0,NaN,65.78,NaN
1665876,1665876,13060240,3.0,4.0,3.0,2252.0,2252.0,6037.0,3.0,33995279.0,...,3101.0,96505.0,0.0,1975.0,402926.0,807716.0,2016.0,404790.0,9249.67,6.037403e+13


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            int64  
 1   parcelid                      int64  
 2   bathroomcnt                   float64
 3   bedroomcnt                    float64
 4   calculatedbathnbr             float64
 5   calculatedfinishedsquarefeet  float64
 6   finishedsquarefeet12          float64
 7   fips                          float64
 8   fullbathcnt                   float64
 9   latitude                      float64
 10  longitude                     float64
 11  lotsizesquarefeet             float64
 12  propertycountylandusecode     object 
 13  propertylandusetypeid         float64
 14  rawcensustractandblock        float64
 15  regionidcity                  float64
 16  regionidcounty                float64
 17  regionidzip                   float64
 18  roomcnt               

In [15]:
# Data dictionary template
numeric_columns = test.select_dtypes(exclude='O')

for column in numeric_columns.columns.to_list():
    print(column, numeric_columns[column].min(), numeric_columns[column].max())

id 0 2985216
parcelid 10711725 169601949
bathroomcnt 0.0 32.0
bedroomcnt 0.0 25.0
calculatedbathnbr 1.0 32.0
calculatedfinishedsquarefeet 1.0 952576.0
finishedsquarefeet12 1.0 427079.0
fips 6037.0 6111.0
fullbathcnt 1.0 32.0
latitude 33324388.0 34819650.0
longitude -119475780.0 -117554316.0
lotsizesquarefeet 100.0 371000512.0
propertylandusetypeid 31.0 279.0
rawcensustractandblock 60371011.101 61110091.003010996
regionidcity 3491.0 396556.0
regionidcounty 1286.0 3101.0
regionidzip 95982.0 399675.0
roomcnt 0.0 96.0
yearbuilt 1801.0 2016.0
structuretaxvaluedollarcnt 1.0 255321161.0
taxvaluedollarcnt 1.0 319622473.0
assessmentyear 2000.0 2016.0
landtaxvaluedollarcnt 1.0 94011079.0
taxamount 0.24 3823175.65
censustractandblock -1.0 483030105084015.0
